In [4]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter, GammaGammaFitter
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [5]:
#Genel bakış.
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [6]:
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.00,12.99,179.33,1.00,2.00,6.00,12.00,80995.00
Price,397885.00,3.12,22.10,0.00,1.25,1.95,3.75,8142.75
Customer ID,397885.00,15294.42,1713.14,12346.00,13969.00,15159.00,16795.00,18287.00


In [7]:
def outlier_tresholds(dataframe, variable):
    quartile_1 = dataframe[variable].quantile(0.01)
    quartile_3 = dataframe[variable].quantile(0.99)
    interquartile = quartile_3 - quartile_1
    low_limit = quartile_1 - 1.5 * interquartile
    up_limit = quartile_3 + 1.5 * interquartile
    return low_limit, up_limit

def replace_tresholds(dataframe, variable):
    low_limit, up_limit = outlier_tresholds(dataframe, variable)
    dataframe[dataframe[variable] > up_limit] = up_limit
    dataframe[dataframe[variable] < low_limit] = low_limit

In [8]:
replace_tresholds(df, "Quantity")
replace_tresholds(df, "Price")

In [9]:
df["total_price"] = df["Quantity"] * df["Price"]

In [10]:
today_date = dt.datetime(2011, 12, 11)
df["InvoiceDate"] = df["InvoiceDate"].apply(pd.to_datetime)

In [11]:
df["InvoiceDate"] = df["InvoiceDate"].apply(pd.to_datetime)

In [12]:
cltv_df = df.groupby("Customer ID").agg({"InvoiceDate": [lambda x: (x.max()-x.min()).days,
                                        lambda x: (today_date-x.min()).days],
                                        "Invoice": lambda x: x.nunique(),
                                        "Price": lambda x: x.sum()})
cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ["recency", "T", "frequency", "monetary"]
cltv_df["recency"] = cltv_df['recency'] / 7
cltv_df["T"] = cltv_df['T'] / 7
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
cltv_df = cltv_df[cltv_df["frequency"] > 1]
cltv_df = cltv_df.reset_index()

In [13]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency"],
        cltv_df["T"])
cltv_df["exp_purch_6_months"] = bgf.predict(4*6,
                                            cltv_df["frequency"],
                                            cltv_df["recency"],
                                            cltv_df["T"]
                                                        )

In [14]:
ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv_df["frequency"],
        cltv_df["monetary"])
cltv_df["exp_ave_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                         cltv_df["monetary"])

In [15]:
cltv_6 = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01
                                   )
cltv_df["cltv_6"] = cltv_6
cltv_df["country"] = df["Country"]
cltv_df = cltv_df[cltv_df['country'] == "United Kingdom"]

In [16]:
ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv_df["frequency"],
        cltv_df["monetary"])
cltv_df["exp_ave_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                         cltv_df["monetary"])

In [17]:
cltv_1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time=1,
                                   freq="W",
                                   discount_rate=0.01
                                   )

In [18]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency"],
        cltv_df["T"])
cltv_df["exp_purch_12_months"] = bgf.predict(4*12,
                                            cltv_df["frequency"],
                                            cltv_df["recency"],
                                            cltv_df["T"]
                                                        )

In [19]:
ggf = GammaGammaFitter(penalizer_coef=0.001)
ggf.fit(cltv_df["frequency"],
        cltv_df["monetary"])
cltv_df["exp_ave_profit"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                         cltv_df["monetary"])

In [20]:
cltv_12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency"],
                                   cltv_df["T"],
                                   cltv_df["monetary"],
                                   time=12,
                                   freq="W",
                                   discount_rate=0.01
                                   )

In [21]:
cltv_df["cltv_1"] = cltv_1
cltv_df["cltv_12"] = cltv_12
cltv_df["country"] = df["Country"]
cltv_df = cltv_df[cltv_df['country'] == "United Kingdom"]

In [22]:
cltv_df.sort_values("cltv_1", ascending=False).head(10)
cltv_df.sort_values("cltv_12", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,exp_purch_6_months,exp_ave_profit,cltv_6,country,exp_purch_12_months,cltv_1,cltv_12
833,14096.00,13.86,14.57,17,1262.76,16.64,1260.41,21940.24,United Kingdom,32.01,3884.77,40990.51
212,12748.00,53.14,53.43,206,52.93,75.14,52.96,4169.68,United Kingdom,147.22,724.27,7929.94
1091,14606.00,53.14,53.43,93,80.76,34.35,80.80,2907.94,United Kingdom,67.31,505.15,5532.49
1429,15311.00,53.29,53.43,91,65.51,33.63,65.57,2310.72,United Kingdom,65.92,401.40,4396.21
563,13521.00,1.14,1.57,3,263.66,8.48,262.75,2326.27,United Kingdom,16.03,422.53,4275.51
1052,14527.00,52.29,52.86,55,94.69,20.78,94.75,2063.43,United Kingdom,40.75,358.53,3927.30
366,13089.00,52.29,52.86,97,51.31,36.06,51.37,1941.02,United Kingdom,70.68,337.22,3692.52
864,14156.00,51.57,53.14,54,86.94,20.24,87.01,1844.83,United Kingdom,39.69,320.52,3512.91
2204,16984.00,5.86,18.71,2,712.76,2.39,703.19,1754.05,United Kingdom,4.72,310.85,3375.33
448,13263.00,46.14,46.57,35,107.44,14.94,107.53,1682.61,United Kingdom,29.25,292.86,3198.61


In [23]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv_6"], 4, labels=["D", "C", "B", "A"])

In [24]:
cltv_df[["segment", 'cltv_6']].groupby("segment").agg(["mean", "count"])

cltv_6      
          mean count
segment             
D        40.45   436
C       121.40   435
B       228.12   435
A       614.60   436